In [1]:
from tsl.datasets import AirQuality
import pickle 
import numpy as np
import pandas as pd
from os import path
from deepecho import PARModel




dataset = AirQuality(impute_nans=True, small=True)
df3 = dataset.dataframe()[:8736]


ts_name = np.arange(0, 52).reshape(-1, 1).repeat(168, 1).reshape(-1)
df3.columns = df3.columns.droplevel('channels')
df3.columns = [str(col) for col in df3.columns]

cols = list(df3.columns)
datetime = df3.index
df3 = pd.DataFrame(df3)
df3.columns = cols
df3.insert(0, 'TS', ts_name)
df3.insert(0, 'Date', datetime)

In [2]:
df3

,Date,TS,1001,1002,1003,1004,1005,1006,1007,1008,...,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036
datetime,,,,,,,,,,,,,,,,,,,,,
2014-05-01 01:00:00,2014-05-01 01:00:00,0,138.0,89.0,105.0,98.0,109.0,87.0,88.0,91.0,...,101.0,84.0,117.0,21.333334,97.0,87.0,74.0,94.0,112.0,109.0
2014-05-01 02:00:00,2014-05-01 02:00:00,0,124.0,85.0,121.0,107.0,101.0,99.0,105.0,102.0,...,100.0,77.0,109.0,78.000000,97.0,84.0,84.0,101.0,123.0,114.0
2014-05-01 03:00:00,2014-05-01 03:00:00,0,127.0,88.0,130.0,115.0,102.0,109.0,114.0,108.0,...,103.0,90.0,105.0,77.000000,103.0,83.0,100.0,112.0,143.0,126.0
2014-05-01 04:00:00,2014-05-01 04:00:00,0,129.0,100.0,137.0,123.0,108.0,118.0,118.0,109.0,...,110.0,94.0,105.0,90.000000,107.0,88.0,103.0,120.0,138.0,130.0
2014-05-01 05:00:00,2014-05-01 05:00:00,0,119.0,109.0,144.0,129.0,115.0,124.0,130.0,116.0,...,105.0,80.0,104.0,83.000000,111.0,85.0,108.0,125.0,145.0,137.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-04-29 20:00:00,2015-04-29 20:00:00,51,150.0,159.0,152.0,135.0,125.0,135.0,131.0,135.0,...,115.0,113.0,127.0,123.000000,131.0,108.0,86.0,99.0,108.0,127.0
2015-04-29 21:00:00,2015-04-29 21:00:00,51,148.0,133.0,64.5,121.0,123.0,55.5,66.0,124.0,...,84.5,127.0,124.0,46.000000,130.0,112.0,99.0,114.0,106.0,130.0
2015-04-29 22:00:00,2015-04-29 22:00:00,51,134.0,126.0,124.0,123.0,118.0,116.0,119.0,117.0,...,122.0,133.0,143.0,147.000000,151.0,121.0,104.0,126.0,101.0,126.0


In [6]:
# Learn a model from the data
if path.exists('pam_airquality2.pkl'):
    with open('pam_airquality2.pkl', 'rb') as f:
        model = pickle.load(f)
else:
    model = PARModel(epochs=100, cuda=True)
    model.fit(
        data=df3,
        segment_size=24,
        entity_columns=['TS'],
        sequence_index='Date'
    )
    with open('pam_airquality2.pkl','wb') as f:
        pickle.dump(model,f)

Loss (0.057): 100%|██████████| 100/100 [06:41<00:00,  4.01s/it]


In [4]:
data_synth = model.sample(num_entities=10)

100%|██████████| 10/10 [00:12<00:00,  1.25s/it]


In [5]:
data_synth

,TS,1001,1002,1003,1004,1005,1006,1007,1008,1009,...,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036
0,0,61.008440,74.560807,82.233703,27.586543,95.832443,110.461038,130.283158,20.224996,94.068172,...,59.466492,-20.338734,67.822003,5.061370,31.523355,31.513430,70.972825,130.189002,97.367472,16.311418
1,0,103.991191,106.732591,108.827352,85.573113,87.634614,64.251263,14.405510,89.619294,29.789164,...,69.816291,20.856269,63.123352,43.705032,53.701730,65.713749,22.053639,-5.510677,97.367472,200.843095
2,0,1.796645,124.481878,71.326061,85.573113,28.469481,81.755069,42.077945,84.114900,83.425303,...,82.431205,68.703224,26.954812,115.644517,73.697310,65.298693,92.980720,133.766417,117.216096,-4.791814
3,0,64.280758,106.798768,1.527196,136.461728,95.832443,75.316786,115.689188,4.861676,30.835696,...,38.941785,44.941219,81.237089,10.736031,62.019219,107.187664,37.419510,54.090040,96.538124,78.638716
4,0,99.456826,74.560807,30.103267,107.095947,16.774275,44.175136,79.271339,80.261754,30.984853,...,129.137537,80.675477,31.053814,73.390298,62.019219,73.156532,118.151293,104.411261,93.359769,103.344934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1675,9,34.435317,72.419394,37.220322,85.573113,88.691784,51.047574,82.331088,71.049738,80.742540,...,37.595529,15.407653,71.047515,80.876323,49.691015,46.428283,53.289123,65.241784,86.743336,101.528932
1676,9,62.811172,102.629176,50.783937,85.573113,83.079033,65.930051,67.650121,104.051979,66.920338,...,33.213868,42.368728,67.822003,45.019199,64.126127,76.762836,31.444749,156.656391,97.844151,71.788665
1677,9,97.789981,86.440504,77.667779,91.450500,72.104373,54.619156,79.296441,71.223391,59.555606,...,67.460316,34.571460,39.961641,23.357788,76.419639,42.358874,46.876692,102.245666,2.366941,117.677733
1678,9,83.384726,45.112854,87.559694,83.904743,139.334677,87.369834,82.373414,76.705622,65.016729,...,60.391840,25.697255,6.752114,76.501198,17.921811,103.886932,81.874443,42.782860,162.229058,165.979601


In [ ]:
# data_synth.to_csv('./synteticAirQualityPam.csv', index=False)